# MNIST classification tutorial

This tutorial wants to show how the scattering transform can be successfully used in classification for the MNIST data base. The results and parameters were set following the table 4 of the paper :

   - Bruna, J., Mallat, S. 'Invariant Scattering Convolutional Networks'. IEEE Transactions on PAMI, 2012.


In [1]:
#Load data and compute the scattering transform of each image
from load_mnist_db import load_scattering_multiresolution_mnist
from keras.datasets import mnist
num_images = 60000
px=32

Xtrain,ytrain,Xtest,ytest = load_scattering_multiresolution_mnist(num_images = num_images, px=px,J=3,L=6,m=2)
Xtrain_1d = Xtrain.reshape((len(Xtrain),-1))
num_samples,num_features = Xtrain_1d.shape



Loading images (in BW!!):
X_train shape: (60000, 32, 32)
60000 train samples
10000 test samples
60000  images loaded in  2.8638360500335693  secs
shape data: (60000, 32, 32)
Create filters:
Compute  60000  scatterings:
0 / 60000
500 / 60000
1000 / 60000
1500 / 60000
2000 / 60000
2500 / 60000
3000 / 60000
3500 / 60000
4000 / 60000
4500 / 60000
5000 / 60000
5500 / 60000
6000 / 60000
6500 / 60000
7000 / 60000
7500 / 60000
8000 / 60000
8500 / 60000
9000 / 60000
9500 / 60000
10000 / 60000
10500 / 60000
11000 / 60000
11500 / 60000
12000 / 60000
12500 / 60000
13000 / 60000
13500 / 60000
14000 / 60000
14500 / 60000
15000 / 60000
15500 / 60000
16000 / 60000
16500 / 60000
17000 / 60000
17500 / 60000
18000 / 60000
18500 / 60000
19000 / 60000
19500 / 60000
20000 / 60000
20500 / 60000
21000 / 60000
21500 / 60000
22000 / 60000
22500 / 60000
23000 / 60000
23500 / 60000
24000 / 60000
24500 / 60000
25000 / 60000
25500 / 60000
26000 / 60000
26500 / 60000
27000 / 60000
27500 / 60000
28000 / 60000
28500 /

In [2]:
Xtrain_1d = Xtrain.reshape((len(Xtrain),-1))
Xtest_1d = Xtest.reshape((len(Xtest),-1))
print('Feature vector:')
Xtrain_1d.shape

(10000, 8128)

In [3]:
import numpy as np

def gethomogeneus_datast(X,y,d,n):
    num_per_class = np.int(n/d)
    ytrain=np.reshape(y,(y.shape[0],))

    X_out = []#np.zeros(n,X.shape[1],X.shape[2],X.shape[3])
    y_out = []
    for i_d in np.arange(d):
        indx = np.where(ytrain.ravel()==i_d)

        X_out.append(X[indx[0][0:num_per_class],:])
        y_out.append(ytrain[indx[0][0:num_per_class],])

    
    X_out = np.concatenate(X_out,axis=0)
    y_out = np.concatenate(y_out,axis=0)
    #print(X_out.shape)
    #print(y_out)
    return X_out,y_out


In [13]:
gammavec = 10**np.arange(-3,-1,0.1)
gammavec.shape = (gammavec.shape[0])
gammavec.shape
Cvec = np.arange(1,3.5,0.5)
Cvec.shape=(Cvec.shape[0])
Cvec.shape

(5,)

In [17]:
##### Gaussian kernel Grid Search
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, Normalizer

# - define range of parameters to check in the crossvalidation
gammavec = 10**np.arange(0.,0.15,0.05) #0.3-1.5(0.25) got 1.995 #(-1,0.5,0.1) got 0.5
gammavec.shape = (gammavec.shape[0])
Cvec = np.arange(4.1,4.4,0.1) #3-5(0.25))got 4.25  #4-5, 4.0 (0.5)
Cvec.shape=(Cvec.shape[0])
parameters = {"C":Cvec,
              "gamma":gammavec}

gs_gaussian = GridSearchCV(SVC(kernel='rbf'),parameters)
pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)

# - find the best parameters with a subset of the training set (otherwise too long to compute)
n = 10000
Xa,ya=gethomogeneus_datast(Xtrain_1d,ytrain,10,n)
pip_gaussian.fit(Xa,ya.ravel())

bestC = gs_gaussian.best_params_['C']
bestgamma = gs_gaussian.best_params_['gamma']
print('Best gamma found:',bestgamma)
print('Best C found:',bestC)


1.1220184543
4.1


In [1]:
########### Evaluate for different number of instances in the traning data (following Table 4 of paper)

num_items,num_features = Xtrain_1d.shape
ns = [300, 1000, 2000, 5000, 10000,20000, 40000, 60000]
score_gaussian = np.zeros((len(ns),1))
print('Evaluate score for different number of training samples: ')

for i,n in enumerate(ns):
    gs_gaussian = SVC(kernel='rbf',C=bestC,gamma=bestgamma)
    pip_gaussian = make_pipeline(MinMaxScaler((-1,1)),Normalizer(),gs_gaussian)

    Xa,ya=gethomogeneus_datast(Xtrain_1d,ytrain,10,n) 
    pip_gaussian.fit(Xa,ya)
    out=pip_gaussian.predict(Xtest_1d)
    score_gaussian[i] = 1.0-accuracy_score(ytest, out)
    print(n,' samples   socer:', score_gaussian[i])
    

NameError: name 'Xtrain_1d' is not defined